In [4]:
import os
import sys

links = ['https://drive.google.com/drive/folders/138z7UB0n-xlwrPP0yh1RL5q1a-ijOGNN',
         'https://drive.google.com/drive/folders/13thmylEd1kh1Fy3W3QhEK_T97EqOEghs',
         'https://drive.google.com/drive/folders/1-BkjKwB6pQQr5I0g3bFx9qvQ3bewZWqI',
         'https://drive.google.com/drive/folders/1CPG_zgU5iqv4v6F-e7hNuDHoitcE5b3Q']

In [2]:
%cd NewVideos

e:\Project\Pose\NewVideos


In [5]:
import gdown

for url in links:
    gdown.download_folder(url, quiet=True, use_cookies=False)

In [8]:
%cd ..

e:\Project\Pose


In [9]:
import cv2
import os

input_folder = "NewVideos"
output_folder = "newpics"

os.makedirs(output_folder, exist_ok=True)


for class_name in os.listdir(input_folder):
    class_path = os.path.join(input_folder, class_name)
    if not os.path.isdir(class_path):
        continue 

    output_class_path = os.path.join(output_folder, class_name)
    os.makedirs(output_class_path, exist_ok=True)


    for video_name in os.listdir(class_path):
        video_path = os.path.join(class_path, video_name)
        if not video_name.endswith(('.mp4', '.avi', '.mov', '.mkv')): 
            continue

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Không thể mở video: {video_path}")
            continue
        video_base_name = os.path.splitext(video_name)[0]

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_filename = f"{output_class_path}/{video_base_name}_frame_{frame_count:04d}.jpg"
            cv2.imwrite(frame_filename, frame)
            frame_count += 1

        cap.release()
        print(f"Đã xử lý video: {video_path}, lưu {frame_count} frames vào {output_class_path}")

print("Hoàn tất xử lý tất cả video!")

Đã xử lý video: NewVideos\False\20241210_172540.mp4, lưu 9167 frames vào newpics\False
Đã xử lý video: NewVideos\False\20241210_173847.mp4, lưu 9156 frames vào newpics\False
Đã xử lý video: NewVideos\False\video_20241209_181558.mp4, lưu 11287 frames vào newpics\False
Đã xử lý video: NewVideos\False\video_20241209_183021.mp4, lưu 7582 frames vào newpics\False
Đã xử lý video: NewVideos\False\video_20241209_184608.mp4, lưu 8497 frames vào newpics\False
Đã xử lý video: NewVideos\False\video_20241210_171248.mp4, lưu 9199 frames vào newpics\False
Đã xử lý video: NewVideos\False\video_20241210_173907.mp4, lưu 7592 frames vào newpics\False
Đã xử lý video: NewVideos\False\VIDEO_DOWNLOAD_1733747152664_1733940797995.mp4, lưu 10159 frames vào newpics\False
Đã xử lý video: NewVideos\False\VIDEO_DOWNLOAD_1733747152908_1733940783383.mp4, lưu 9084 frames vào newpics\False
Đã xử lý video: NewVideos\False\VIDEO_DOWNLOAD_1733747153229_1733940770998.mp4, lưu 11428 frames vào newpics\False
Đã xử lý video: 

In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [2]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

In [3]:
import json
def extract_keypoints(image_path, confidence_threshold=0.5):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        keypoints = []
        for lm in result.pose_landmarks.landmark:
            keypoints.append([lm.x, lm.y, lm.visibility])
        return keypoints
    return None

In [4]:
def process_folder(input_dir, output_dir):
    """
    Process all images in a folder and save extracted keypoints as JSON files.

    Args:
        input_dir (str): Path to the folder containing images.
        output_dir (str): Path to the folder to save JSON files.
    """
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(input_dir, file_name)
            keypoints = extract_keypoints(image_path)
            if keypoints:
                json_path = os.path.join(output_dir, file_name.replace('.jpg', '.json'))
                with open(json_path, 'w') as f:
                    json.dump(keypoints, f)
            else:
                print(f"Skipping {file_name}, no keypoints detected.")
                os.remove(image_path)

In [ ]:
import os
import cv2
base_dir = r"E:\Project\Pose\newpics"
output_dir = r"E:\Project\Pose\keypoints_dataset_new"
for label in ["True", "False"]:
    input_path = os.path.join(base_dir, label)
    output_path = os.path.join(output_dir, label)
    process_folder(input_path, output_path)